In [15]:
import os

In [7]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\Dell\\Downloads\\Rent_Ml_flow_AWS\\Rent_Prediction_Using_ML'

In [16]:
import pandas as pd

In [17]:
data = pd.read_csv("artifacts/data_ingestion/delhi_num_rental.csv")
data.head()

,size_sq_ft,bedrooms,latitude,longitude,closest_metro_station_km,price
0,1050,2,28.594969,77.298668,0.417142,20000
1,2250,2,28.641806,77.293922,0.125136,28000
2,1350,2,28.644363,77.293228,0.371709,28000
3,450,2,28.594736,77.311150,1.087760,12500
4,600,2,28.592464,77.305161,0.523298,10000


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12660 entries, 0 to 12659
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   size_sq_ft                12660 non-null  int64  
 1   bedrooms                  12660 non-null  int64  
 2   latitude                  12660 non-null  float64
 3   longitude                 12660 non-null  float64
 4   closest_metro_station_km  12660 non-null  float64
 5   price                     12660 non-null  int64  
dtypes: float64(3), int64(3)
memory usage: 593.6 KB


In [19]:
data.isnull().sum()

size_sq_ft                  0
bedrooms                    0
latitude                    0
longitude                   0
closest_metro_station_km    0
price                       0
dtype: int64

In [20]:
data.shape

(12660, 6)

In [21]:
data.columns

Index(['size_sq_ft', 'bedrooms', 'latitude', 'longitude',
       'closest_metro_station_km', 'price'],
      dtype='object')

In [30]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [34]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [35]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [36]:
import os
from src.mlProject import logger

In [37]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    # Checks presence of all required columns using schema, does not check for dtype
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e



In [38]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-01-16 14:39:08,643: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-16 14:39:08,659: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-16 14:39:08,719: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-16 14:39:08,769: INFO: common: created directory at: artifacts]
[2025-01-16 14:39:08,815: INFO: common: created directory at: artifacts/data_validation]
